In [2]:
import os
import glob
import subprocess
import pandas as pd

# Filter sequences

In [3]:
input="tlr4-ago1"
best_binders=pd.read_csv(f"output/{input}/af2_best.csv")
all_binders=pd.read_csv(f"output/{input}/af2_results.csv")
best_binders

,mpnn,plddt,i_ptm,i_pae,rmsd,seq,model_path,binder-rmsd,input_pdb
0,1.213537,0.775261,0.691565,11.134387,1.760293,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_4H_9_1/af2...,4.963032,output/tlr4-ago1/rf_dock/tlr4-ago1_4H_9_1/thre...
1,1.160942,0.766465,0.602153,13.715751,2.000111,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_4H_9_1/af2...,4.809887,output/tlr4-ago1/rf_dock/tlr4-ago1_4H_9_1/thre...
2,1.300598,0.689482,0.670574,12.757600,2.933392,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_lcb3_1/af2...,3.063305,output/tlr4-ago1/rf_dock/tlr4-ago1_lcb3_1/thre...
3,1.140846,0.817809,0.763596,9.682425,2.175548,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_3H_3_1/af2...,7.905189,output/tlr4-ago1/rf_dock/tlr4-ago1_3H_3_1/thre...
4,1.097666,0.811251,0.718439,11.075122,1.965440,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_3H_3_1/af2...,8.154563,output/tlr4-ago1/rf_dock/tlr4-ago1_3H_3_1/thre...
5,1.060904,0.862328,0.787758,8.596767,2.863401,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_3H_3_1/af2...,8.001517,output/tlr4-ago1/rf_dock/tlr4-ago1_3H_3_1/thre...
6,1.167340,0.830703,0.562138,14.819692,2.920332,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_3H_3_1/af2...,7.801794,output/tlr4-ago1/rf_dock/tlr4-ago1_3H_3_1/thre...
7,1.111934,0.835997,0.597583,13.953725,2.636881,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_3H_3_1/af2...,7.792325,output/tlr4-ago1/rf_dock/tlr4-ago1_3H_3_1/thre...


In [4]:
filtered = best_binders[(best_binders["rmsd"]<3)&(best_binders["binder-rmsd"]<4)]
#[(best_binders["plddt"]>0.7)&(best_binders["i_pae"]<8)&(best_binders["rmsd"]<3)]

filtered = filtered.sort_values(by='plddt', ascending=False)
filtered

,mpnn,plddt,i_ptm,i_pae,rmsd,seq,model_path,binder-rmsd,input_pdb
2,1.300598,0.689482,0.670574,12.7576,2.933392,RLHKLTLRNNFDSLNVMKTCIQGLAGLEVHRLVLGEFRNEGNLEKF...,output/tlr4-ago1/mpnn_af2/tlr4-ago1_lcb3_1/af2...,3.063305,output/tlr4-ago1/rf_dock/tlr4-ago1_lcb3_1/thre...


# Prepare folder of best binders

In [5]:
# Create output dir
binder_folder="input"
output_dir=f"output/{input}/opt_binders"
binders_input=f"{output_dir}/{binder_folder}"
os.makedirs(f"{binders_input}", exist_ok=True)

# Copy binder inputs to new dir
pdbs=filtered["model_path"].to_list()
for pdb in pdbs:
    !cp $pdb $binders_input

# Count .pdb files in the folder
pdb_files = glob.glob(os.path.join(binders_input, "*.pdb"))
pdb_count = len(pdb_files)
print(f"Number of binders to optimize: {pdb_count}. Will run {pdb_count} arrays")

Number of binders to optimize: 1. Will run 1 arrays


# Run optimization for each binder in parallel

In [7]:

# Binder optimization parameters
iterations=20
starting_mpnns=200

designs_per_iteration=10
mpnn_per_design=10
sort="i_pae"

af2_recycles=12
mpnn_sampling_temp=0.15

array_number=pdb_count
array_limit=10

# Script arguments
bash_arguments=f"--array=0-{array_number-1}%{array_limit}"
script_arguments=f"{binders_input} {output_dir} {iterations} {starting_mpnns} {designs_per_iteration} {mpnn_per_design} {sort} {af2_recycles} {mpnn_sampling_temp}"

command = f"sbatch {bash_arguments} helper_scripts/binder_opt.sh {script_arguments}"

print(f"command will run {array_number} binder optimizations, each doing")
print(f"{iterations} iterations with {designs_per_iteration*mpnn_per_design} sampled designs.")
print(f"Each script will return {iterations*designs_per_iteration*mpnn_per_design} designs ({iterations*designs_per_iteration*mpnn_per_design*array_number} in total)")
print(command)

command will run 1 binder optimizations, each doing
20 iterations with 100 sampled designs.
Each script will return 2000 designs (2000 in total)
sbatch --array=1-1%10 helper_scripts/binder_opt.sh output/tlr4-ago1/opt_binders/input output/tlr4-ago1/opt_binders 20 200 10 10 i_pae 12 0.15


In [13]:
# Run the array bash script
subprocess.run(command, shell=True)

Submitted batch job 144921


CompletedProcess(args='sbatch --array=1-1%10 helper_scripts/binder_opt.sh output/tlr4-ago1/opt_binders/input output/tlr4-ago1/opt_binders 20 200 10 10 i_pae 12 0.15', returncode=0)

In [12]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
    144761_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144762_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144763_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144764_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144765_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144766_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144767_[2-5%1]       gpu binder_d  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144775_[2-5%1]       gpu tlr4-ant  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144776_[2-5%1]       gpu tlr4-ant  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144777_[2-5%1]       gpu tlr4-ant  tsatler PD       0:00      1 (JobArrayTaskLimit)
    144778_[2-5%1]       gpu tlr4-a